In [64]:
import uiautomator2 as u2

d = u2.connect()

In [65]:
d.info

{'currentPackageName': 'com.android.systemui',
 'displayHeight': 2232,
 'displayRotation': 0,
 'displaySizeDpX': 360,
 'displaySizeDpY': 780,
 'displayWidth': 1080,
 'productName': 'YAL-AL10',
 'screenOn': False,
 'sdkInt': 29,
 'naturalOrientation': True}

## 锁屏解锁
自动识别息屏、亮屏状态并解锁, 支持无密码、数字、图案

In [61]:
def unlock(d):
    info = d.info
    if info.get('screenOn') and info.get('currentPackageName') != 'com.android.systemui':
        # 屏幕已点亮且当前包名不是系统UI（意味着已经在主屏幕或者应用内）
        pass
    elif info.get('screenOn') and info.get('currentPackageName') == 'com.android.systemui':
        # 屏幕已点亮且当前包名是系统UI（可能在锁屏界面）
        d.press('recent')  # 可能会唤醒密码输入界面
        # 模拟解锁手势
        d.swipe_points([(250, 1270), (555, 1570), (850, 1890), (865, 1570), (535, 1300), (260, 1565), (265, 1870), (850, 1275)], 0.1)
        d.click(0.9, 0.5)
    else:
        # 屏幕未点亮
        d.screen_on()
        d.press('recent')
        # 模拟解锁手势
        d.swipe_points([(250, 1270), (555, 1570), (850, 1890), (865, 1570), (535, 1300), (260, 1565), (265, 1870), (850, 1275)], 0.1)
        d.click(0.9, 0.5)
        
